In [1]:
!pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
action_list = ["a question", "an answer to a question", "a challenge", "an accusation", "a request for action", "a denial", "an apology", "an acceptance", "a statement", "a proposal", "an appreciation", "a disagreement", "an agreement", "a negative reaction", "a positive reaction"]

SPP_to_question = ["an answer to a question", "a statement"]
SPP_to_accusation = ["a denial", "an admission", "an apology"] 
SPP_to_request = ["a rejection", "a denial", "an acceptance"]
SPP_to_challenge = ["a rejection", "an acceptance", "a denial", "an admission", "an apology"]
SPP_to_proposal = ["a rejection", "a denial", "an acceptance"] 


mismatched_spp_list = []
mismatched_fpp_list = []
mismatched_pairs_records = {}
matched_spp_list = []
matched_fpp_list = []
matched_pairs_records = {}

mismatched_cosines = []
matched_cosines = []


In [3]:


def get_probabilities_of_acts(msg_probs):
    long = {'a question': 0.00, 'an answer to a question': 0.00, 'a challenge': 0.00, 'an accusation': 0.00, 'a request for action': 0.00, 'an admission': 0.00, 'a denial': 0.00, 'an apology': 0.00, 'a rejection': 0.00, 'an acceptance': 0.00, 'a statement': 73.0, 'an announcement': 53.00, 'a proposal': 0.00, 'an appreciation': 0.00, 'a disagreement': 0.00, 'an agreement': 0.00, 'a negative reaction': 0.00, 'a positive reaction': 0.00}
    act_list = ['a question', 'an answer to a question', 'a challenge', 'an accusation', 'a request for action', 'an admission', 'a denial', 'an apology', 'a rejection', 'an acceptance', 'a statement', 'an announcement', 'a proposal', 'an appreciation', 'a disagreement', 'an agreement', 'a negative reaction', 'a positive reaction']
    
    if msg_probs == "LONG":
        probabilities = long
    else:
        probabilities = msg_probs
        
    for act in act_list:
        prob = probabilities[act]
        probab_changed = prob / 100
        probabilities[act] = probab_changed
    return probabilities

def select_main_actions(probabilities):
    long = {'a question': 0.00, 'an answer to a question': 0.00, 'a challenge': 0.00, 'an accusation': 0.00, 'a request for action': 0.00, 'an admission': 0.00, 'a denial': 0.00, 'an apology': 0.00, 'a rejection': 0.00, 'an acceptance': 0.00, 'a statement': 73.0, 'an announcement': 53.00, 'a proposal': 0.00, 'an appreciation': 0.00, 'a disagreement': 0.00, 'an agreement': 0.00, 'a negative reaction': 0.00, 'a positive reaction': 0.00}
    act_list = ['a question', 'an answer to a question', 'a challenge', 'an accusation', 'a request for action', 'an admission', 'a denial', 'an apology', 'a rejection', 'an acceptance', 'a statement', 'an announcement', 'a proposal', 'an appreciation'] #, 'a disagreement', 'an agreement', 'a negative reaction', 'a positive reaction']
    prob_list = []
    if probabilities == "LONG": #NB. This is refers to a comment that was unclassifiable with MNLI 0-shot as its length was over the token max boundary
      probabilities = long

    for act in act_list:
        try:
          prob = probabilities[act]
          prob_list.append((act, prob))
        except:
          print(probabilities)
    sorted_probs = sorted(prob_list, key = lambda x: x[1], reverse = True)
    main_acts_list = []

    
    main_actions_pre = [sorted_probs[0][0], sorted_probs[1][0]]
    i = 0
    for act in main_actions_pre: #we decided to merge these action categories for simplicity, as these were very close to each other.
        if act == "a rejection":
            main_actions_pre[i] = "a denial"
        elif act == "an admission":
            main_actions_pre[i] = "an acceptance"
        elif act == "an announcement":
            main_actions_pre[i] = "a statement"

        else:
            pass
        i += 1
    main_actions = main_actions_pre
    return main_actions

def select_agreement_level(probabilities):
    probabilities = get_probabilities_of_acts(probabilities)
    long = {'a question': 0.00, 'an answer to a question': 0.00, 'a challenge': 0.00, 'an accusation': 0.00, 'a request for action': 0.00, 'an admission': 0.00, 'a denial': 0.00, 'an apology': 0.00, 'a rejection': 0.00, 'an acceptance': 0.00, 'a statement': 73.0, 'an announcement': 53.00, 'a proposal': 0.00, 'an appreciation': 0.00, 'a disagreement': 0.00, 'an agreement': 0.00, 'a negative reaction': 0.00, 'a positive reaction': 0.00}
    act_list = ['a disagreement', 'an agreement']
    prob_list = []
    if probabilities == "LONG":
      probabilities = long

    for act in act_list:
        try:
          prob = probabilities[act]
          prob_list.append((act, prob))
        except:
          print(probabilities)
    sorted_probs = sorted(prob_list, key = lambda x: x[1], reverse = True)
    agr_lev = sorted_probs[0][0]
    if float(sorted_probs[0][1]) - float(sorted_probs[1][1]) > 0.2:
        return agr_lev
    else:
        return "neutral"

def select_reaction_valence(probabilities):
    act_list = ['a negative reaction', 'a positive reaction']
    first = act_list[0]
    second = act_list[1]
    if probabilities[first] < 1:
        pass
    elif probabilities[second] < 1:
        pass
    else:
        probabilities = get_probabilities_of_acts(probabilities)
    long = {'a question': 0.00, 'an answer to a question': 0.00, 'a challenge': 0.00, 'an accusation': 0.00, 'a request for action': 0.00, 'an admission': 0.00, 'a denial': 0.00, 'an apology': 0.00, 'a rejection': 0.00, 'an acceptance': 0.00, 'a statement': 73.0, 'an announcement': 53.00, 'a proposal': 0.00, 'an appreciation': 0.00, 'a disagreement': 0.00, 'an agreement': 0.00, 'a negative reaction': 0.00, 'a positive reaction': 0.00}
    
    prob_list = []
    if probabilities == "LONG":
      probabilities = long

    for act in act_list:
        try:
          prob = probabilities[act]
          prob_list.append((act, prob))
        except:
          print(probabilities)
    sorted_probs = sorted(prob_list, key = lambda x: x[1], reverse = True)
    valence = sorted_probs[0][0]
    if float(sorted_probs[0][1]) - float(sorted_probs[1][1]) > 0.2:
        return valence
    else:
        return "neutral"

In [4]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [5]:


# load model pretrained on MNLI

from transformers import BartForSequenceClassification, BartTokenizer

from transformers import BartForSequenceClassification, BartTokenizer

Bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')


def count_probabilities_of_hypotheses(premise, probabilities):
  # pose sequence as a NLI premise and label (ACTION) as a hypothesis
  hyp_list = ["a question", "an answer to a question", "a challenge", "an accusation", "a request for action", "an admission", "a denial", "an apology", "a rejection", "an acceptance", "a statement", "an announcement", "a proposal", "an appreciation", "a disagreement", "an agreement", "a negative reaction", "a positive reaction"]

  for hyp in hyp_list:
      
    hypothesis = str("This text is " + hyp + ".")
    #print(hypothesis)
    # run through model pre-trained on MNLI
    input_ids = Bart_tokenizer.encode(premise, hypothesis, return_tensors='pt')
    logits = model(input_ids)[0]

    # we throw away "neutral" (dim 1) and take the probability of
    # "entailment" (2) as the probability of the label being true 
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    true_prob = probs[:,1].item() * 100
    #print(f'Probability that the label is true: {true_prob:0.2f}%')

    probabilities[hyp] = true_prob
  
  return probabilities


def get_MNLI_probabilities(item):
  post_text = item["Message_text"]
  post_tokens = tokenizer.tokenize(post_text)
  if len(post_text) > 1500:
    probabilities = "LONG"
    item['Probabilities'] = probabilities
  else:
    probabilities = {}
    try:
      probabilities = count_probabilities_of_hypotheses(post_text, probabilities)
    except:
      probabilities = "LONG"
    #post_probabilities.append((item, probabilities))
    print(post_text, probabilities)
    item['Probabilities'] = probabilities
  return item

In [6]:
'''
Created on 6.11.2020

@author: paakkih1
'''
 
#!pip uninstall transformers
#!pip install transformers==3.5.1

!pip install unidecode
!pip install jsonlines
import nltk
nltk.download('punkt')

from sentence_transformers import SentenceTransformer, util

from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize
import json
from nltk.stem.snowball import SnowballStemmer
from unidecode import unidecode
from prettytable import PrettyTable
#from Calculations import Calculator
import csv
import os
from os import getcwd
import glob
import collections
import jsonlines
import pandas as pd
from nltk.tokenize import sent_tokenize


snowballstem = SnowballStemmer('english')
tokenizer = TweetTokenizer()
prompt_type_list = ["Prompt: Casual", "Prompt: Moderation", "Prompt: Coordination", "Prompt: Contention", "Prompt: Editing", "Prompt: Procedures"]
politeness_features = ["Please", "Please_start", "HASHEDGE", "Indirect_(btw)", "Hedges", "Factuality", "Deference", "Gratitude", "Apologizing", "1st_person_pl.", "1st_person", "1st_person_start", "2nd_person", "2nd_person_start", "Indirect_(greeting)", "Direct_question", "Direct_start", "HASPOSITIVE", "HASNEGATIVE", "SUBJUNCTIVE", "INDICATIVE"]


def lemmatize(post):
    token_sentences = sent_tokenize(post, "english")
    lemma_sentences = []
    for s in token_sentences:
        tokens = tokenizer.tokenize(s)
        for tok in tokens:
            unidecode(tok)
            snowballstem.stem(tok)
            lemma_sentences.append(tok)
    return lemma_sentences

def is_question(reply_to_msg, post_actions):
    if reply_to_msg != None:
        if "a question" in post_actions:
            return True
        elif post_actions == "a question":
            return True
        #elif post_action == "whQuestion":
            #return True
    return False

def is_accusation(reply_to_msg, post_actions):
    if reply_to_msg != None:
        if "an accusation" in post_actions:
            return True
        elif post_actions == "an accusation":
            return True
    return False

def is_request(reply_to_msg, post_actions):
    if reply_to_msg != None:
        if "a request for action" in post_actions:
            return True
        elif post_actions == "a request for action":
            return True
    return False

def is_challenge(reply_to_msg, post_actions):
    if reply_to_msg != None:
        if "a challenge" in post_actions:
            return True
        elif post_actions == "a challenge":
            return True
    return False

def is_proposal(reply_to_msg, post_actions):
    if reply_to_msg != None:
        if "a proposal" in post_actions:
            return True
        elif post_actions == "a proposal":
            return True
    return False

def is_question_type(post_actions):
    if "a question" in post_actions:
        return True
    elif post_actions == "a question":
        return True
    elif post_actions == "whQuestion":
        return True
    else:
        return False

def is_accusation_type(post_actions):
    if "an accusation" in post_actions:
        return True
    elif post_actions == "an accusation":
        return True
    else:
        return False

def is_request_type(post_actions):
    if "a request for action" in post_actions:
        return True
    elif post_actions == "a request for action":
        return True
    else:
        return False

def is_challenge_type(post_actions):
    if "a challenge" in post_actions:
        return True
    elif post_actions == "a challenge":
        return True
    else:
        return False

def acceptable_response(post_actions, fpp_action):
    found = False
    if fpp_action == "a question":
      acceptable_answers = SPP_to_question
    elif fpp_action == "an accusation":
      acceptable_answers = SPP_to_accusation
    elif fpp_action == "a request for action":
      acceptable_answers = SPP_to_request
    elif fpp_action == "a challenge":
      acceptable_answers = SPP_to_challenge
    elif fpp_action == "a proposal":
      acceptable_answers = SPP_to_proposal
    else:
      return True
    for act in post_actions:
      if act in acceptable_answers:
        found = True
        return True
      else:
        found = False
    if found == False:
      return False

def is_normative_fpp(fpp_action):
    normative_fpps = ["a question", "an accusation", "a request for action", "a challenge", "a proposal"]
    if fpp_action in normative_fpps:
      return True
    else:
      return False
    

def check_if_reply_to_question_missing(msg, conversations):

    reply_to = msg["Reply_to_msg_ID"]
    try:
      author_id = msg['Author_ID']
      if author_id == "none":
        author_id = msg['Author']
    except:
      author_id = msg['Author']
      
    post_id = msg['Message_ID']
    conversation_id = msg['Conversation_id']

    targeted_author = None

    ## find to whom this questions was targeted
    for post_msg in conversations[conversation_id]['messages']:
        if reply_to == post_msg['Message_ID']:
            targeted_author = post_msg['Author']

    
    if targeted_author == None:
        return False, targeted_author

    ## check other posts by this user and if there are responses
    loytynyt = False
    i = 0
    still_present = False
    for post_msg in conversations[conversation_id]['messages']:
        if post_msg['Author'] == targeted_author:
            still_present = True
            if post_msg["Reply_to_msg_ID"] == post_id:
                loytynyt = True
            else:
                i += 1
        else:
            pass

    if loytynyt == False:
        if i < 1:
            pass
        else:
            if still_present == True:
                return True, targeted_author

    return False, targeted_author

def check_if_post_asymmetric_with_parent(conversations, reply_to_msg, post_actions, conversation_id, post_text, mismatch_check_matching_post_index, mismatch_check_post_index):
    mismatched = True
    found_fpp = False
    fpp_actions = []
    if reply_to_msg != None:
        for msg in conversations[conversation_id]["messages"]:
            if msg['Message_ID'] == reply_to_msg:
                found_fpp = True
                fpp_actions = msg['action_type']
                for act in fpp_actions:
                    if is_normative_fpp(act) == True:
                        if acceptable_response(post_actions, act) == True:
                            mismatched = False

                    else:
                        mismatched = False
    else:
        mismatched = False

    #add messages of the pairs to lists and record their info for cosine similarity tests:
    if mismatched == True:
        mismatched_spp_list.append(post_text)
        mismatched_fpp_list.append(msg['Message_text'])
        ind = str(mismatch_check_post_index)
        mismatched_pairs_records[ind] = [conversation_id, post_text, msg['Message_text'], 0]
        mismatched_cosines.append([conversation_id, mismatch_check_post_index, 0])
        mismatch_check_post_index += 1

    else:
        if reply_to_msg != None:
            if found_fpp == True:
                matched_spp_list.append(post_text)
                matched_fpp_list.append(msg['Message_text'])
                ind = str(mismatch_check_matching_post_index)
                matched_pairs_records[ind] = [conversation_id, post_text, msg['Message_text'], 0]
                matched_cosines.append([conversation_id, mismatch_check_matching_post_index, 0])
                mismatch_check_matching_post_index += 1

    return mismatched, fpp_actions, mismatch_check_matching_post_index, mismatch_check_post_index

def calculate_cosine_similarities():
    model = SentenceTransformer('paraphrase-distilroberta-base-v1')
    
    #try shortening long posts
    j = 0
    for post in mismatched_fpp_list:
      if len(post) > 1500:
        post = post[:800]
        mismatched_fpp_list[j] = post
        j += 1

    j = 0
    for post in matched_fpp_list:
      if len(post) > 1500:
        post = post[:800]
        matched_fpp_list[j] = post
        j += 1

    j = 0
    for post in mismatched_spp_list:
      if len(post) > 1500:
        post = post[:800]
        mismatched_spp_list[j] = post
        j += 1

    j = 0
    for post in matched_spp_list:
      if len(post) > 1500:
        post = post[:800]
        matched_spp_list[j] = post
        j += 1

    print("length of mismatched list:", len(mismatched_fpp_list))
    print("length of matched list:", len(matched_fpp_list))
    print("length of mismatched cos:", len(mismatched_cosines))
    print("length of matched cos:", len(matched_cosines))

    #Compute embedding for mismatching pairs, both lists
    embeddings_mismatched_fpp = model.encode(mismatched_fpp_list, convert_to_tensor=True)
    embeddings_mismatched_spp = model.encode(mismatched_spp_list, convert_to_tensor=True)

    #Compute cosine-similarits
    try:
        cosine_scores = util.pytorch_cos_sim(embeddings_mismatched_fpp, embeddings_mismatched_spp)
    except:
        pass
    i = 0

    for item in mismatched_fpp_list:
        cosine_scr = cosine_scores[i][i]
        try:
          mismatched_cosines[i][2] = cosine_scr
        except:
          print("failed on index:", i, mismatched_cosines[i][1])
        i += 1

    #Compute embedding for both matching pairs lists
    embeddings_matched_fpp = model.encode(matched_fpp_list, convert_to_tensor=True)
    embeddings_matched_spp = model.encode(matched_spp_list, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores_matching = util.pytorch_cos_sim(embeddings_matched_fpp, embeddings_matched_spp)

    #Output the pairs with their score
    i = 0
    for item in matched_fpp_list:
        cosine_scr = cosine_scores_matching[i][i]
        try:
          matched_cosines[i][2] = cosine_scr
        except:
          print("failed on index:", i, matched_cosines[i][1])
        i += 1

def get_mean_cosine_similarity_for_fpps_and_spps(conversation):
    matched_scores = 0
    mismatched_scores = 0
    nro_matched = 0
    nro_mismatched = 0

    i = 0
    for item in matched_cosines:
        if item[0] == conversation: 
            score = item[2] 
            matched_scores += score
            nro_matched += 1
        i += 1

    j = 0
    for item in mismatched_cosines:
        #ind = str(j)
        if item[0] == conversation: 
            score = item[2] 
            mismatched_scores += score
            nro_mismatched += 1
        j += 1

    if nro_mismatched == 0:
        cosine_mismatched = 0
    else:
        cosine_mismatched = mismatched_scores / nro_mismatched
    if nro_matched == 0:
        cosine_matched = 0
    else:
        cosine_matched = matched_scores / nro_matched
    return cosine_mismatched, cosine_matched, nro_mismatched, nro_matched

def count_negative_words(negative_words, lemmatized_post):
    nro_negs = 0
    for w in lemmatized_post:
        if w in negative_words:
            nro_negs += 1
    if len(lemmatized_post) != 0:
        neg_words_mean = nro_negs / len(lemmatized_post)
    else:
        neg_words_mean = 0
    return nro_negs, neg_words_mean

def count_troll_words(original_post_dict, accuse_users, users):
    lemmatized_post =  lemmatize(original_post_dict['Message_text'])
    author = str(original_post_dict['Author'])
    troll_words = ["troll", "trolling", "troller", "trolls", "trollishly", "trollish", "trolled", "russian-trolls", "Russian trolls",
               "troll's", "trolls'", "paid-for-trolling", "half-troll", "full-troll", "troll-like", "trolling-like", "tricktrolling",
               "ricktrolling", "facts.troll"]
    nro_troll = 0
    trollw = False
    for w in lemmatized_post:
        w = w.lower()
        if w in troll_words:
            nro_troll += 1
            trollw = True
        else:
            for tr in troll_words:
                if tr in w:
                    print(str(w))
    if author in users:
        pass
    else:
        #nro_accuse_users += 1
        users.append(str(author))
    if trollw == True:
        if author in accuse_users:
            pass
        else:
            accuse_users.append(str(author))
    return nro_troll, accuse_users, users


def get_conversation_specific_info(conversations):
    conversations_info = []
    
    toxicity_data = open_data_file("troll_data_utterances_actions_toxicity.json")
    
    #get conversation-specific info for all conversations:
    for conversation in conversations:
        conversation_tox = 0
        conversation_question_count = 0
        conversation_answer_count = 0
        conversation_negreact_count = 0
        conversation_appre_count = 0
        conversation_dis_count = 0
        conversation_state_count = 0
        admission_count = 0
        reject_count = 0
        positive_react_count = 0
        agree_count = 0
        apology_count = 0
        accept_count = 0
        accuse_count = 0
        request_count = 0
        challenge_count = 0
        denial_count = 0
        announcement_count = 0
        appreciation_count = 0
        proposal_count = 0
        neutral_agr = 0
        neutral_valence = 0
        
        troll_mention_count = 0
        troll_accusing_users = 0
        list_accuse_users = []
        list_users = []
        
        label = conversations[conversation]["label"]
        miss_reps = conversations[conversation]["missing_reps"]
        mismatched_reps = conversations[conversation]["mismatched_reps"]
        negative_word_count = conversations[conversation]["negative_words"]
        mismatch_question = conversations[conversation]["mismatched_reps_to_questions"]
        mismatch_reps_to_accusations = conversations[conversation]["mismatched_reps_to_accusations"]
        mismatched_reps_to_requests = conversations[conversation]["mismatched_reps_to_requests"]
        mismatched_reps_to_challenges = conversations[conversation]["mismatched_reps_to_challenges"]
        mismatched_reps_to_proposals = conversations[conversation]["mismatched_reps_to_proposals"]
        expected_reps = conversations[conversation]["expected_reps"]
        expected_reps_to_questions = conversations[conversation]["expected_reps_to_questions"]
        expected_reps_to_accusations = conversations[conversation]["expected_reps_to_accusations"]
        expected_reps_to_requests = conversations[conversation]["expected_reps_to_requests"]
        expected_reps_to_challenges = conversations[conversation]["expected_reps_to_challenges"]
        expected_reps_to_proposals = conversations[conversation]["expected_reps_to_proposals"]
        missing_reps_to_questions = conversations[conversation]["missing_reps_to_questions"]
        missing_reps_to_accusations = conversations[conversation]["missing_reps_to_accusations"]
        missing_reps_to_requests = conversations[conversation]["missing_reps_to_requests"]
        missing_reps_to_challenges = conversations[conversation]["missing_reps_to_challenges"]
        missing_reps_to_proposals = conversations[conversation]["missing_reps_to_proposals"]
        nonmissing_reps = conversations[conversation]["nonmissing_reps"]
        reps_to_questions = conversations[conversation]["reps_to_questions"]
        reps_to_accusations = conversations[conversation]["reps_to_accusations"]
        reps_to_requests = conversations[conversation]["reps_to_requests"]
        reps_to_challenges = conversations[conversation]["reps_to_challenges"]
        reps_to_proposals = conversations[conversation]["reps_to_proposals"]

        #for act in list_of_actions_probs:
        prob_question = conversations[conversation]["a question"]
        prob_answer_to_a_question = conversations[conversation]["an answer to a question"]
        prob_challenge = conversations[conversation]["a challenge"]
        prob_accusation = conversations[conversation]["an accusation"]
        prob_request = conversations[conversation]["a request for action"]
        prob_admission = conversations[conversation]["an admission"]
        prob_denial = conversations[conversation]["a denial"]
        prob_apology = conversations[conversation]["an apology"]
        prob_rejection = conversations[conversation]["a rejection"]
        prob_acceptance = conversations[conversation]["an acceptance"]
        prob_statement = conversations[conversation]["a statement"]
        prob_announcement = conversations[conversation]["an announcement"]
        prob_proposal = conversations[conversation]["a proposal"]
        prob_appreciation = conversations[conversation]["an appreciation"]
        prob_disagreement = conversations[conversation]["a disagreement"]
        prob_agreement = conversations[conversation]["an agreement"]
        prob_negative_reaction = conversations[conversation]["a negative reaction"]
        prob_positive_reaction = conversations[conversation]["a positive reaction"]

        prompt_polite = open_data_file("troll_data_prompts_and_politeness.json")
        
        i = 0
        for prompt in prompt_type_list:
            conversations[conversation][prompt] = prompt_polite[conversation][prompt]
            i += 1

        i = 0
        for polite in politeness_features:
            conversations[conversation][polite] = prompt_polite[conversation][polite]
            i += 1
        
        for msg in conversations[conversation]["messages"]:
            original_id = msg['Message_ID']
            try:
                if msg["Toxicity"] == -9999.0:
                  pass
                elif float(msg["Toxicity"]) == -9999.0:
                  pass
                else:
                  conversation_tox += msg["Toxicity"]
            except:
                for post_item in toxicity_data[conversation]['messages']:
                    if original_id == post_item['Message_ID']:
                        try:
                            toxicity = post_item['Toxicity']
                        except:
                            pass
                        try:
                            if float(toxicity) == -9999.0:
                                pass
                            else:
                                conversation_tox += float(toxicity)
                        except:
                            print(toxicity)
                
            
            if "a question" in msg["action_type"]:
                conversation_question_count += 1
            elif is_question_type(msg["action_type"]) == True:
                conversation_question_count += 1
            if "an answer to a question" in msg["action_type"]:
                conversation_answer_count += 1
            if "an admission" in msg["action_type"]:
                admission_count += 1
            if "a negative reaction" in msg["action_type"]: 
                conversation_negreact_count += 1
            if "an appreciation" in msg["action_type"]: 
                conversation_appre_count += 1
            if "a statement" in msg["action_type"]: 
                conversation_state_count += 1
            if "a rejection" in msg["action_type"]: 
                reject_count += 1
            if "a positive reaction" in msg["action_type"]: 
                positive_react_count += 1
            if "an agreement" in msg["action_type"]: 
                agree_count += 1
            if "an apology" in msg["action_type"]: 
                apology_count += 1
            if "an acceptance" in msg["action_type"]:
                accept_count += 1
            if "an accusation" in msg["action_type"]:
                accuse_count += 1
            if "a request for action" in msg["action_type"]:
                request_count += 1
            if "a challenge" in msg["action_type"]:
                challenge_count += 1
            if "a denial" in msg["action_type"]:
                denial_count += 1
            if "an announcement" in msg["action_type"]:
                announcement_count += 1
            if "a proposal" in msg["action_type"]: 
                proposal_count += 1


            agreement_type = select_agreement_level(msg['Probabilities'])
            if agreement_type == "a disagreement": 
                conversation_dis_count += 1
            elif agreement_type == "an agreement":
                agree_count += 1
            else:
                neutral_agr += 1

            valence_type = select_reaction_valence(msg['Probabilities'])
            if valence_type == "a negative reaction":
                conversation_negreact_count += 1
            elif valence_type == "a positive reaction":
                positive_react_count += 1
            else:
                neutral_valence += 1
            
            troll_mentions, list_accuse_users, list_users = count_troll_words(msg, list_accuse_users, list_users)
            troll_mention_count += troll_mentions

        message_count = len(conversations[conversation]["messages"])
        if message_count > 0:
          conversation_tox_mean = conversation_tox / message_count
          conversation_neg_mean = negative_word_count / message_count
        else:
          conversation_tox_mean = 0
          conversation_neg_mean = 0
        
        av_message_length =  conversations[conversation]["average_message_length"] / message_count
        
        mean_cosine_similarity_fpps_and_spps_mismatched, mean_cosine_similarity_fpps_and_spps_matched, nro_of_mismatched, nro_of_matched = get_mean_cosine_similarity_for_fpps_and_spps(conversation)
        mean_cosine_similarity_fpps_and_spps = (mean_cosine_similarity_fpps_and_spps_mismatched + mean_cosine_similarity_fpps_and_spps_matched) / 2
        
        troll_accusing_users = len(list_accuse_users)
        nro_users = len(list_users)
        
        conversations_info.append((miss_reps, mismatched_reps, (mismatched_reps + miss_reps), nro_of_matched, negative_word_count, conversation_neg_mean, conversation_tox, conversation_tox_mean, message_count, av_message_length, conversation_question_count, conversation_answer_count, admission_count, announcement_count, denial_count, reject_count, challenge_count, request_count, proposal_count, accuse_count, apology_count, accept_count, positive_react_count, agree_count, conversation_negreact_count, conversation_dis_count, conversation_appre_count, conversation_state_count, mismatch_question, mismatch_reps_to_accusations, mismatched_reps_to_requests, mismatched_reps_to_challenges, mismatched_reps_to_proposals, expected_reps, expected_reps_to_questions, expected_reps_to_accusations, expected_reps_to_requests, expected_reps_to_challenges, expected_reps_to_proposals, missing_reps_to_questions, missing_reps_to_accusations, missing_reps_to_requests, missing_reps_to_challenges, missing_reps_to_proposals, nonmissing_reps, reps_to_questions, reps_to_accusations, reps_to_requests, reps_to_challenges, reps_to_proposals, (prob_question / 100), (prob_answer_to_a_question / 100), (prob_challenge / 100), (prob_accusation / 100), (prob_request / 100), (prob_admission / 100), (prob_denial / 100), (prob_apology / 100), (prob_rejection / 100), (prob_acceptance / 100), (prob_statement / 100), (prob_announcement / 100), (prob_proposal / 100), (prob_appreciation / 100), (prob_disagreement / 100), (prob_agreement / 100), (prob_negative_reaction / 100), (prob_positive_reaction / 100), mean_cosine_similarity_fpps_and_spps_mismatched, mean_cosine_similarity_fpps_and_spps_matched, mean_cosine_similarity_fpps_and_spps, label, conversations[conversation]['filename'], conversations[conversation]['branch_ID'], conversations[conversation]["messages"][0], neutral_agr, neutral_valence, troll_mention_count, troll_accusing_users, nro_users,
                                    conversations[conversation]["Prompt: Casual"], conversations[conversation]["Prompt: Moderation"], conversations[conversation]["Prompt: Coordination"], conversations[conversation]["Prompt: Contention"], conversations[conversation]["Prompt: Editing"], conversations[conversation]["Prompt: Procedures"],
                                    conversations[conversation]["Please"], conversations[conversation]["Please_start"], conversations[conversation]["HASHEDGE"], conversations[conversation]["Indirect_(btw)"], conversations[conversation]["Hedges"], conversations[conversation]["Factuality"], conversations[conversation]["Deference"], 
                                    conversations[conversation]["Gratitude"], conversations[conversation]["Apologizing"], conversations[conversation]["1st_person_pl."], conversations[conversation]["1st_person"], conversations[conversation]["1st_person_start"], conversations[conversation]["2nd_person"], 
                                    conversations[conversation]["2nd_person_start"], conversations[conversation]["Indirect_(greeting)"], conversations[conversation]["Direct_question"], conversations[conversation]["Direct_start"], conversations[conversation]["HASPOSITIVE"], conversations[conversation]["HASNEGATIVE"], conversations[conversation]["SUBJUNCTIVE"], conversations[conversation]["INDICATIVE"]))
        
    return conversations_info


def get_negatives():
    with open('negative-words.txt', "r") as txt_file:
        negatives = txt_file.readlines()
        txt_file.close()
    negative_words = []
    for w in negatives:
        n_word = w.strip()
        negative_words.append(n_word)
    f_word = negative_words[0][3:]
    negative_words.append(f_word)
    return negative_words

def open_data_file(filename):
    with open(filename, "r") as f:
        data = json.load(f)
        f.close()
    return data

def record_number_of_replies_to_author(reply_to_msg, conversations, authors, conversation_id):
    author = None
    for msg in conversations[conversation_id]["messages"]:
        if msg['Message_ID'] == reply_to_msg:
            author = msg['Author']
    if reply_to_msg != None:
        if author != None:
            authors[author]["nro_replies"] += 1
    else:
        print("No author found.")
    return authors

#def get_prompt_types(post_id):
#    id_found = False
#    data = pd.read_csv('troll_prompt_types.csv', header=0)
#    for i in range(len(data)):
#      for line in data:
#          line_2 = data[line][i]
#          line_2 = line_2.split(" ")
#          if str(post_id) == str(line_2[0]):
#                prompts = line_2[1:]
#                id_found = True
#          else:
#                pass
#    if id_found == False:
#      print("ID not found", post_id)
#      prompts = ['0', '0', '0', '0', '0', '0']
#    return prompts

#def get_politeness_strategies(post_id):
#    id_found = False
#    data = pd.read_csv('troll_politeness_strategies.csv', header=0)
#    for i in range(len(data)):
##      for line in data:
 #         line_2 = data[line][i]
#          line_2 = line_2.split(" ")
#          if str(post_id) == str(line_2[0]):
#              politeness_s = line_2[1:]
#              id_found = True
#          else:
#              pass
#    if id_found == False:
#      print("ID not found", post_id)
#      politeness_s = ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
#    return politeness_s


def process_all_posts_in_data(data, negative_words, authors, conversations, questions, accusations, requests, challenges, proposals):
    list_of_actions_probs = ["a question", "an answer to a question", "a challenge", "an accusation", "a request for action", "an admission", "a denial", "an apology", "a rejection", "an acceptance", "a statement", "an announcement", "a proposal", "an appreciation", "a disagreement", "an agreement", "a negative reaction", "a positive reaction"]
    mismatch_check_matching_post_index = 0
    mismatch_check_post_index = 0

    for conv_id in data:
      try:
        posts = data[conv_id]['messages']
      except:
        print(conv_id)
        pass
      for post in posts:
        author = post['Author']
        conversation_id = post['Conversation_id']
        reply_to_msg = post['Reply_to_msg_ID']
        action_probabilities = post['Probabilities']
        no_id = False
        try:
            post_id = post['Message_ID']
            if post_id == "none":
                no_id = True
                post_id = post['Author']
        except:
            post_id = post['Author']
        try:
            post_actions = post['action_type']
        except:
            probs = get_probabilities_of_acts(action_probabilities)
            post_actions = select_main_actions(probs)
            post['action_type'] = post_actions
        date_time = post['Date_and_time']
        msg_text = post['Message_text']
        lemmatized_post = lemmatize(msg_text)
        
        #get the number of negative words used in the post:
        nro_of_negative_words, neg_words_mean = count_negative_words(negative_words, lemmatized_post)
        authors[author]['messages'][post_id]['neg_words'] = nro_of_negative_words
        authors[author]['negative_words'] += nro_of_negative_words
        conversations[conversation_id]["negative_words"] += nro_of_negative_words
        conversations[conversation_id]["average_message_length"] += len(msg_text)
        
        #record the probabilities of different actions:
        for act in list_of_actions_probs:
            probab = action_probabilities[act]
            conversations[conversation_id][act] += probab

        #record number of replies-to-certain-author:
        if reply_to_msg != None:
            authors = record_number_of_replies_to_author(reply_to_msg, conversations, authors, conversation_id)
        
        #check if post is a reply to another user's post, not the article, and if the post is asymmetric with the parent post action (boolean):
        mismatched, fpp_actions, mismatch_check_matching_post_index, mismatch_check_post_index = check_if_post_asymmetric_with_parent(conversations, reply_to_msg, post_actions, conversation_id, msg_text, mismatch_check_matching_post_index, mismatch_check_post_index)
        
        #then add +1 to mismatched count if true, per author and per conversation:
        if mismatched == True:
            authors[author]['mismatched_reps'] += 1
            conversations[conversation_id]["mismatched_reps"] += 1
            if "a question" in fpp_actions:
              authors[author]['mismatched_reps_to_questions'] += 1
              conversations[conversation_id]["mismatched_reps_to_questions"] += 1
            if "an accusation" in fpp_actions:
              authors[author]['mismatched_reps_to_accusations'] += 1
              conversations[conversation_id]["mismatched_reps_to_accusations"] += 1
            if "a request for action" in fpp_actions:
              authors[author]['mismatched_reps_to_requests'] += 1
              conversations[conversation_id]["mismatched_reps_to_requests"] += 1
            if "a challenge" in fpp_actions:
              authors[author]['mismatched_reps_to_challenges'] += 1
              conversations[conversation_id]["mismatched_reps_to_challenges"] += 1
            if "a proposal" in fpp_actions:
              authors[author]['mismatched_reps_to_proposals'] += 1
              conversations[conversation_id]["mismatched_reps_to_proposals"] += 1
            
        else:
            authors[author]['expected_reps'] += 1
            conversations[conversation_id]["expected_reps"] += 1
            if "a question" in fpp_actions:
              authors[author]['expected_reps_to_questions'] += 1
              conversations[conversation_id]["expected_reps_to_questions"] += 1
            if "an accusation" in fpp_actions:
              authors[author]['expected_reps_to_accusations'] += 1
              conversations[conversation_id]["expected_reps_to_accusations"] += 1
            if "a request for action" in fpp_actions:
              authors[author]['expected_reps_to_requests'] += 1
              conversations[conversation_id]["expected_reps_to_requests"] += 1
            if "a challenge" in fpp_actions:
              authors[author]['expected_reps_to_challenges'] += 1
              conversations[conversation_id]["expected_reps_to_challenges"] += 1
            if "a proposal" in fpp_actions:
              authors[author]['expected_reps_to_proposals'] += 1
              conversations[conversation_id]["expected_reps_to_proposals"] += 1

    #author_missing_responses_count = collections.defaultdict( int )

    #Count all missing answers per author and per conversation:
    list_of_fpps = [questions, accusations, requests, challenges, proposals]

    j = 0
    for fpp_list in list_of_fpps:
      for msg in fpp_list:
          conversation_id = msg["Conversation_id"]
          missing, missing_response_author = check_if_reply_to_question_missing(msg, conversations)
          if missing:
              authors[missing_response_author]["missing_reps"] += 1
              conversations[conversation_id]["missing_reps"] += 1
              if j == 0:
                authors[author]['missing_reps_to_questions'] += 1
                conversations[conversation_id]["missing_reps_to_questions"] += 1
              elif j == 1:
                authors[author]['missing_reps_to_accusations'] += 1
                conversations[conversation_id]["missing_reps_to_accusations"] += 1
              elif j == 2:
                authors[author]['missing_reps_to_requests'] += 1
                conversations[conversation_id]["missing_reps_to_requests"] += 1
              elif j == 3:
                authors[author]['missing_reps_to_challenges'] += 1
                conversations[conversation_id]["missing_reps_to_challenges"] += 1
              elif j == 4:
                authors[author]['missing_reps_to_proposals'] += 1
                conversations[conversation_id]["missing_reps_to_proposals"] += 1
          else:
              try:
                authors[missing_response_author]["nonmissing_reps"] += 1
              except:
                pass
              conversations[conversation_id]["nonmissing_reps"] += 1
              if j == 0:
                authors[author]['reps_to_questions'] += 1
                conversations[conversation_id]["reps_to_questions"] += 1
              elif j == 1:
                authors[author]['reps_to_accusations'] += 1
                conversations[conversation_id]["reps_to_accusations"] += 1
              elif j == 2:
                authors[author]['reps_to_requests'] += 1
                conversations[conversation_id]["reps_to_requests"] += 1
              elif j == 3:
                authors[author]['reps_to_challenges'] += 1
                conversations[conversation_id]["reps_to_challenges"] += 1
              elif j == 4:
                authors[author]['reps_to_proposals'] += 1
                conversations[conversation_id]["reps_to_proposals"] += 1
      j += 1

    for conv in conversations:
      nro_msgs = len(conversations[conv]["messages"])
      for act in list_of_actions_probs:
        prob = conversations[conv][act]
        conversations[conv][act] = prob / nro_msgs
    return authors, conversations

def get_corresponding_post_count(messages, category):
    count = 0
    for msg in messages:
        if category == "Reply_to_msg_ID":
            try:
              if messages[msg][category] == "Original article":
                  count += 1
            except:
              pass
        else:
            pass
    return count


def write_into_csv(data, filename):
    i = 0
    with open(filename, mode='w') as troll_file:
        troll_writer = csv.writer(troll_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for line in data:
            try:
                troll_writer.writerow(line)
            except:
                try:
                    print(line[0])
                except:
                    line[0] = str("user" + str(i))
                    troll_writer.writerow(line)
            i += 1
    troll_file.close()


def main():
    '''
    First get negative words to use in negative word use analysis. NB. negative word file is a list of blacklist or offensive/taboo words from previous research (Liu et al., 2005).
    Then get comment data for each file in specified path, and get user-specific data per file.
    '''
    negative_words = get_negatives()
    
    
    #record all conversation ids and create a dictionary of conversations
    #NB. the troll data file cannot be openly published due to privacy issues. We are planning to make the data available to researchers via application, if possible with regard to privacy and ethical issues.
    
    filen = "troll_conversations_branches_trolls.json"
    conv_data = open_data_file(filen)
    
    #make archives to record info specifically about conversations, authors and posts classified as questions:
    list_of_actions_probs = ["a question", "an answer to a question", "a challenge", "an accusation", "a request for action", "an admission", "a denial", "an apology", "a rejection", "an acceptance", "a statement", "an announcement", "a proposal", "an appreciation", "a disagreement", "an agreement", "a negative reaction", "a positive reaction"]
    
    conversations = {}
    authors = {}
    question_data_archive = []
    accusation_data_archive = []
    request_data_archive = []
    challenge_data_archive = []
    proposal_data_archive = []
    
    for item in conv_data:
        print(item)
        conv_id = item
        try:
            conv_info = conversations[conv_id]
            
        except:
            conversations[conv_id] = {}
            label = conv_data[item]["label"]

            conv_filename = conv_data[item]['filename']
            trollname = conv_data[item]['trollname']
            branch_ID = conv_data[item]['branch_ID']

            conversations[conv_id]["filename"] = conv_filename
            conversations[conv_id]["trollname"] = trollname
            conversations[conv_id]["branch_ID"] = branch_ID
            conversations[conv_id]["label"] = label
            conversations[conv_id]["messages"] = []
            conversations[conv_id]["missing_reps"] = 0
            conversations[conv_id]["mismatched_reps"] = 0
            conversations[conv_id]["negative_words"] = 0
            conversations[conv_id]["mismatched_reps_to_questions"] = 0
            conversations[conv_id]["mismatched_reps_to_accusations"] = 0
            conversations[conv_id]["mismatched_reps_to_requests"] = 0
            conversations[conv_id]["mismatched_reps_to_challenges"] = 0
            conversations[conv_id]["mismatched_reps_to_proposals"] = 0
            conversations[conv_id]["expected_reps"] = 0
            conversations[conv_id]["expected_reps_to_questions"] = 0
            conversations[conv_id]["expected_reps_to_accusations"] = 0
            conversations[conv_id]["expected_reps_to_requests"] = 0
            conversations[conv_id]["expected_reps_to_challenges"] = 0
            conversations[conv_id]["expected_reps_to_proposals"] = 0
            conversations[conv_id]["missing_reps_to_questions"] = 0
            conversations[conv_id]["missing_reps_to_accusations"] = 0
            conversations[conv_id]["missing_reps_to_requests"] = 0
            conversations[conv_id]["missing_reps_to_challenges"] = 0
            conversations[conv_id]["missing_reps_to_proposals"] = 0
            conversations[conv_id]["nonmissing_reps"] = 0
            conversations[conv_id]["reps_to_questions"] = 0
            conversations[conv_id]["reps_to_accusations"] = 0
            conversations[conv_id]["reps_to_requests"] = 0
            conversations[conv_id]["reps_to_challenges"] = 0
            conversations[conv_id]["reps_to_proposals"] = 0
            conversations[conv_id]["average_message_length"] = 0
            conversations[conv_id]["toxicity"] = 0
            
            for prompt in prompt_type_list:
                conversations[conv_id][prompt] = 0

            for polite in politeness_features:
                conversations[conv_id][polite] = 0

            
            for act in list_of_actions_probs:
                conversations[conv_id][act] = 0

    #Get post data for conversations:
    
    with open('troll_conversations_branches_trolls.json', "r") as f:
        data = json.load(f)
        f.close()
        count = 2887
        
        for conv_id in data:
            line = data[conv_id]
            j = 0
            for post in line["messages"]:
              try:
                post_id = post["Message_ID"] 
              except:
                print(post)
                
              #Get the main action types:
              try:
                action_probabilities = post["Probabilities"]
              except:
                #print(conv_id)
                #print(post)
                post = get_MNLI_probabilities(post)
                #print(post)

              if action_probabilities == "LONG":
                action_probabilities = {'a question': 0.00, 'an answer to a question': 0.00, 'a challenge': 0.00, 'an accusation': 0.00, 'a request for action': 0.00, 'an admission': 0.00, 'a denial': 0.00, 'an apology': 0.00, 'a rejection': 0.00, 'an acceptance': 0.00, 'a statement': 73.0, 'an announcement': 53.00, 'a proposal': 0.00, 'an appreciation': 0.00, 'a disagreement': 0.00, 'an agreement': 0.00, 'a negative reaction': 0.00, 'a positive reaction': 0.00}
                
                
              dialogue_class = select_main_actions(action_probabilities)
              post["action_type"] = dialogue_class
              if data[conv_id]["messages"][j]["Message_ID"] == post_id:
                data[conv_id]["messages"][j]["action_type"] = dialogue_class
                data[conv_id]["messages"][j]["Probabilities"] = action_probabilities
                data[conv_id]["messages"][j]["Conversation_id"] = conv_id

              classified_post = post
              conversation = conv_id
              post["Conversation_id"] = conv_id
              post_text = post["Message_text"]
              author = post["Author"]
              #meta_info = line["meta"]
              reply_to = post["Reply_to_msg_ID"]
              timestamp = post["Date_and_time"]
              actions = post["action_type"]

              #add all questions and others actions to their designated list:
              if is_question(reply_to, actions) == True:
                  question_data_archive.append(post)
              if is_accusation(reply_to, actions) == True:
                  accusation_data_archive.append(post)
              if is_request(reply_to, actions) == True:
                  request_data_archive.append(post)
              if is_challenge(reply_to, actions) == True:
                  challenge_data_archive.append(post)
              if is_proposal(reply_to, actions) == True:
                  proposal_data_archive.append(post)

              #add posts info to corresponding conversation dictionary:
              conversations[conversation]["messages"].append(post)

              #add author data to author dictionary
              try:
                  auth = authors[author]
                  auth["messages"][post_id] = post

              except:
                  authors[author] = {}
                  authors[author]["messages"] = {}
                  authors[author]["messages"][post_id] = post


                  authors[author]["nro_replies"] = 0
                  authors[author]["missing_reps"] = 0
                  authors[author]["mismatched_reps"] = 0
                  authors[author]["negative_words"] = 0
                  authors[author]["toxicity"] = 0
                  authors[author]["personal_attacks"] = 0
                  authors[author]["headers"] = 0
                  authors[author]["mismatched_reps_to_questions"] = 0
                  authors[author]["non_headers"] = 0
                  authors[author]["mismatched_reps_to_accusations"] = 0
                  authors[author]["mismatched_reps_to_requests"] = 0
                  authors[author]["mismatched_reps_to_challenges"] = 0
                  authors[author]["mismatched_reps_to_proposals"] = 0
                  authors[author]["expected_reps"] = 0
                  authors[author]["expected_reps_to_questions"] = 0
                  authors[author]["expected_reps_to_accusations"] = 0
                  authors[author]["expected_reps_to_requests"] = 0
                  authors[author]["expected_reps_to_challenges"] = 0
                  authors[author]["expected_reps_to_proposals"] = 0
                  authors[author]["missing_reps_to_questions"] = 0
                  authors[author]["missing_reps_to_accusations"] = 0
                  authors[author]["missing_reps_to_requests"] = 0
                  authors[author]["missing_reps_to_challenges"] = 0
                  authors[author]["missing_reps_to_proposals"] = 0
                  authors[author]["nonmissing_reps"] = 0
                  authors[author]["reps_to_questions"] = 0
                  authors[author]["reps_to_accusations"] = 0
                  authors[author]["reps_to_requests"] = 0
                  authors[author]["reps_to_challenges"] = 0
                  authors[author]["reps_to_proposals"] = 0

                  authors[author]["a question"] = 0
                  authors[author]["an answer to a question"] = 0
                  authors[author]["a challenge"] = 0
                  authors[author]["an accusation"] = 0
                  authors[author]["a request for action"] = 0
                  authors[author]["an admission"] = 0
                  authors[author]["a denial"] = 0
                  authors[author]["an apology"] = 0
                  authors[author]["a rejection"] = 0
                  authors[author]["an acceptance"] = 0
                  authors[author]["a statement"] = 0
                  authors[author]["an announcement"] = 0
                  authors[author]["a proposal"] = 0
                  authors[author]["an appreciation"] = 0
                  authors[author]["a disagreement"] = 0
                  authors[author]["an agreement"] = 0
                  authors[author]["a negative reaction"] = 0
                  authors[author]["a positive reaction"] = 0
              j += 1

            
    authors, conversations = process_all_posts_in_data(data, negative_words, authors, conversations, question_data_archive, accusation_data_archive, request_data_archive, challenge_data_archive, proposal_data_archive)
    
    #calculate cosine similarities between fpps and spps:
    calculate_cosine_similarities()

    #calculate "awriness score" for each author
    author_troll_probability = get_author_specific_values(authors)

    #Count conversation-specific info:
    conversations_info = get_conversation_specific_info(conversations)

    write_into_csv(conversations_info, "troll_conversations_features_16122023.csv")



main()


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt to /home/paakkih1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Guardian10_prospect-boris-johnson-prime-minister-reignited-labour-civil-war_updated_action_probabilities.jsonbranch0
Guardian10_prospect-boris-johnson-prime-minister-reignited-labour-civil-war_updated_action_probabilities.jsonbranch1
Guardian10_prospect-boris-johnson-prime-minister-reignited-labour-civil-war_updated_action_probabilities.jsonbranch2
Guardian10_prospect-boris-johnson-prime-minister-reignited-labour-civil-war_updated_action_probabilities.jsonbranch3
Guardian10_prospect-boris-johnson-prime-minister-reignited-labour-civil-war_updated_action_probabilities.jsonbranch4
Guardian10_prospect-boris-johnson-prime-minister-reignited-labour-civil-war_updated_action_probabilities.jsonbranch5
Guardian10_prospect-boris-johnson-prime-minister-reignited-labour-civil-war_updated_action_probabilities.jsonbranch6
Guardian10_prospect-boris-johnson-prime-minister-reignited-labour-civil-war_updated_action_probabilities.jsonbranch7
Guardian10_prospect-boris-johnson-prime-minister-reignited-labou

length of mismatched list: 8326
length of matched list: 48067
length of mismatched cos: 8326
length of matched cos: 48067
<class 'dict'>
controlling
controlling
controlled
controlled
controlled
controlled
controlling
controlling
uncontrolled
uncontrolled
controlled
controlled
uncontrollably
controlled
controlled
controlled
controlled
stroll
stroll
highly-controlled
highly-controlled
controlling
controlling
controlled
controlled
comtrolled
comtrolled
controlled
controlled
controlling
controlling
controlled
controlled
controlled
controlled
controlling
controlling
controlling
controlling
controlling
controlling
controlling
controlling
controlled
controlled
controlling
controlling
uncontrolled
uncontrolled
well-controlled
well-controlled
controlled
controlled
controlling
controlling
controlling
controlling
controlled
controlled
controlled
controlled
uncontrolled
uncontrolled
controlled
controlled
controlled
controlled
controlling
controlling
controlling
controlling
controlling
controlling
